<a href="https://colab.research.google.com/github/lucas-fpaiva/estatistica_pes/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import random as python_random
import tensorflow as tf

#IA
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import GRU

import warnings
warnings.filterwarnings('ignore')
% matplotlib inline

#Funções

In [2]:
def reset_seeds():
   np.random.seed(4) 
   python_random.seed(9498)
   tf.random.set_seed(9)

reset_seeds() 

def data_import(txt):
  global columns
  columns=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21"]
  df = pd.read_csv("/content/drive/MyDrive/cmap/"+txt, sep= "\s+", header = None,names=columns )
  return df

def add_rul(g):
    g['RUL'] = max(g['cycle']) - g['cycle']
    return g

def add_cycle2(g):
    g['cycle2'] = 362 - g['RUL']
    return g

def new_data(train_data):
        new_data = train_data[["id","cycle","op1","op2","op3","RUL"]]
        new_data['sensor']=[columns[5]]*train_data.shape[0]
        new_data['sensor_value']= train_data[columns[5]]
        for column in columns[5:]:
            temp_data = train_data[["id","cycle","op1","op2","op3","RUL"]]
            temp_data['sensor']=[column]*train_data.shape[0];
            temp_data['sensor_value']= train_data[column];
            new_data = pd.concat([new_data,temp_data],ignore_index=True)
        return new_data 

def Suavisar(train_data, mm):
        new_data = train_data[["id","cycle","op1","op2","op3","RUL"]]
        new_data['sensor_value']= train_data[columns[5]]
        for column in columns[5:]:
            temp_data = train_data[["id","cycle","op1","op2","op3","RUL"]]
            temp_data['sensor']=[column]*train_data.shape[0];
            temp_data['sensor_value']= train_data[column];
            new_data = pd.concat([new_data,temp_data],ignore_index=True)
        return new_data 



#Dados

In [3]:
train_FD001 = data_import("train_FD001.txt") 
train_FD002 = data_import("train_FD002.txt") 
train_FD003 = data_import("train_FD003.txt") 
train_FD004 = data_import("train_FD004.txt") 

train_FD001 = train_FD001.groupby('id').apply(add_rul)
train_FD002 = train_FD002.groupby('id').apply(add_rul)
train_FD003 = train_FD003.groupby('id').apply(add_rul)
train_FD004 = train_FD004.groupby('id').apply(add_rul)

train_FD001_2 = train_FD001.groupby('id').apply(add_cycle2)
train_FD002_2 = train_FD002.groupby('id').apply(add_cycle2)
train_FD003_2 = train_FD003.groupby('id').apply(add_cycle2)
train_FD004_2 = train_FD004.groupby('id').apply(add_cycle2)

new_train_FD001 = new_data(train_FD001) 
new_train_FD002 = new_data(train_FD002) 
new_train_FD003 = new_data(train_FD003) 
new_train_FD004 = new_data(train_FD004) 

In [4]:
from statistics import mean, median, mode, stdev, variance

In [5]:
max_list = []
def add_rul(g):
    global max_list
    max_list.append(max(g['cycle']))
    return g

r = train_FD001.groupby('id').apply(add_rul)

#Parte2

In [6]:
np.mean(np.array(max_list))

206.31

In [7]:
df2 = train_FD001_2[train_FD001_2[ 'cycle2' ] > 206]

In [8]:
max_list = []
def add_rul(g):
    global max_list
    max_list.append(max(g['cycle']+1-min(g['cycle'])))
    return g

r = df2.groupby('id').apply(add_rul)

In [9]:
Data = df2[0:156][["sensor2","sensor3","sensor4","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor17"
         ,"sensor20","sensor21"]].rolling(25).mean()
k = 156

for i in max_list[1:]:
    data = df2[k:i+k][["sensor2","sensor3","sensor4","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor17"
         ,"sensor20","sensor21"]].rolling(25).mean()
    Data = Data.append(data)

    k =  k+i

#df2["sensor2"][0:192] = np.array([0,0])

In [10]:
new_data = df2[["id","cycle2","op1","op2","op3","RUL"]]
new_data.reset_index(drop=True, inplace=True)
Data.reset_index(drop=True, inplace=True)
Data_S = pd.concat([new_data,Data], axis = 1)
Data_S = Data_S.dropna(axis=0, subset=['sensor2'])

In [12]:
Data_S['sensor4'].isna().sum()

0

#Modelo

In [13]:
df2=Data_S

In [14]:
#X = df2.drop(['RUL',"sensor14","sensor10","sensor9","sensor8","op3","op2","op1",'id', "cycle2"], axis = 1) 
#X = train.drop(["RUL"], axis = 1) 
X =df2[["sensor2","sensor3","sensor15","sensor17"]]
y = df2['RUL']
X=X.astype('float32')
y=y.astype('float32')

X_train = X 
y_train = y

In [15]:
X_train.shape

(13081, 4)

In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1)) 
X_train = scaler.fit_transform(X_train) 

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
model = Sequential()
n= 256 
n2 = 256 
n3 =  128
n4 =  256 
n5=512
model.add(Dense(3,activation='LeakyReLU', input_shape=(4,)))
model.add(Dense(n,activation='LeakyReLU'))
#model.add(Dropout(0.2))
model.add(Dense(n2,activation='LeakyReLU'))
model.add(Dense(n3,activation='LeakyReLU'))
model.add(Dropout(0.2))
model.add(Dense(n4,activation='LeakyReLU'))
model.add(Dense(n5,activation='LeakyReLU'))
#model.add(Dropout(0.2))
model.add(Dense(1,activation='LeakyReLU'))
model.compile(optimizer='adam', loss='mae')

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
reset_seeds()
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model.fit(x=X_train,y=y_train.values,
          validation_split=0.1,
          batch_size=1024,epochs=90, callbacks=[early_stop])

Epoch 1/90
12/12 [==============================] - 1s 57ms/step - loss: 63.8007 - val_loss: 57.4958
Epoch 2/90
12/12 [==============================] - 1s 45ms/step - loss: 43.3321 - val_loss: 26.7382
Epoch 3/90
12/12 [==============================] - 1s 42ms/step - loss: 21.4620 - val_loss: 21.9155
Epoch 4/90
12/12 [==============================] - 1s 42ms/step - loss: 19.6668 - val_loss: 19.1348
Epoch 5/90
12/12 [==============================] - 0s 41ms/step - loss: 18.4625 - val_loss: 18.0473
Epoch 6/90
12/12 [==============================] - 0s 41ms/step - loss: 18.2958 - val_loss: 18.1710
Epoch 7/90
12/12 [==============================] - 1s 43ms/step - loss: 18.1350 - val_loss: 18.0248
Epoch 8/90
12/12 [==============================] - 1s 71ms/step - loss: 18.0655 - val_loss: 17.7793
Epoch 9/90
12/12 [==============================] - 1s 64ms/step - loss: 18.0939 - val_loss: 18.0786
Epoch 10/90
12/12 [==============================] - 1s 46ms/step - loss: 18.0672 - val_los

#Teste

In [19]:
columns=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21"]


test_data = pd.read_csv("/content/drive/MyDrive/cmap/test_FD001.txt", sep= "\s+", header = None,names=columns)

In [20]:
max_list = []
def add_rul(g):
    global max_list
    max_list.append(max(g['cycle']))
    return g

r = test_data.groupby('id').apply(add_rul)

In [21]:
Data = test_data[0:31][["sensor2","sensor3","sensor4","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor17"
         ,"sensor20","sensor21"]].rolling(25).mean()
k = 31

for i in max_list[1:]:
    data = test_data[k:i+k][["sensor2","sensor3","sensor4","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor17"
         ,"sensor20","sensor21"]].rolling(25).mean()
    Data = Data.append(data)

    k =  k+i

In [22]:
val = []
s=0
for i in max_list:
  s = s+i
  val.append(s-1)


In [23]:
X = test_data.drop(["sensor19","sensor18","sensor16","sensor14","sensor10","sensor9","sensor6","sensor5","sensor1","op3","op2","op1",'id', "cycle"], axis = 1) 
X = Data[["sensor2","sensor3","sensor15","sensor17"]]
X=X.astype('float32')
from sklearn.model_selection import train_test_split 

X_test = X 
X_test = scaler.fit_transform(X_test) 


In [24]:
x_test = []
for i in val:
  x_test.append(X_test[i])
x_test = np.array([x_test])

In [25]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error,mean_absolute_error
from sklearn.metrics import r2_score
predictions = model.predict(x_test[0])
#mean_absolute_error(y_test,predictions)

In [26]:
p=predictions

In [27]:
y_test = pd.read_csv("/content/drive/MyDrive/cmap/RUL_FD001.txt", sep= "\s+", header = None)

In [28]:
y_test = y_test.values
p=np.reshape(p, len(p))
y=np.array(y_test)
y=np.reshape(y_test, len(y_test))

#Resultados

In [29]:
mean_absolute_error(y,p)

27.553237840943037

In [30]:
mean_absolute_error(y,p)

27.553237840943037

In [31]:
mean_absolute_error(y[1:2],p[1:2])

30.973678588867188

In [32]:
MAE = []
RMSE = []
for i in range(100):
  MAE.append(mean_absolute_error(y[0+i:i+1],p[0+i:i+1]))
  RMSE.append(np.sqrt(mean_squared_error(y[0+i:i+1],p[0+i:i+1])))


In [33]:
RMSE_new = np.sqrt(mean_squared_error(y,p))
print("RMSE (Taking only last examples): ", RMSE_new)

RMSE (Taking only last examples):  34.94960352171008


In [34]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
#p,y_test = zip(*sorted(zip(p,y_test)))
fig = go.Figure()
fig.add_trace(go.Scatter(y=p,name="previsão"))#,mode='markers',marker_symbol="x"))
fig.add_trace(go.Scatter(y=y,name="Dados"))#,mode='markers'))
fig.update_layout(title='Dados vs Previsão',
                   xaxis_title='Unidade',
                   yaxis_title='RUL')
fig.show()

In [35]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
#p,y_test = zip(*sorted(zip(p,y_test)))
fig = go.Figure()
fig.add_trace(go.Scatter(x=y,y=p,mode='markers'))
fig.update_layout(title='Dados vs Previsão',
                   xaxis_title='Dados',
                   yaxis_title='Previsão',
                  width=700,height=700)
fig.show()

In [36]:
model.save("MLP.h5")